In [ ]:
print("\n Global speed distribution analysis")
blueTrajs, redTraj = get_trajs(nDrops, red_particle_idx, rawTrajs)
bin_borders = np.arange(0, 100, .2)
bin_centers = np.arange(0, 100, .2)[:-1] + .2 / 2
x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 10000)

v_blue = ys.speed_ensemble(blueTrajs, step = v_step)
v_red = ys.speed_ensemble(redTraj, step = v_step)
T_blue, T_blue_std = fit_hist(v_blue, bin_borders, MB_2D, [1.])
T_red, T_red_std = fit_hist(v_red, bin_borders, MB_2D, [1.])
print("Raw Trajectories")
print(f"Blue Particles σ: {T_blue[0]} ± {T_blue_std[0]}")
print(f"Red Particle σ: {T_red[0]} ± {T_red_std[0]}")

if smooth_verb:
    blueTrajs_smooth, redTraj_smooth = get_trajs(nDrops, red_particle_idx, smoothTrajs) 
    v_blue_smooth = ys.speed_ensemble(blueTrajs_smooth, step = v_step)
    v_red_smooth = ys.speed_ensemble(redTraj_smooth, step = v_step)
    T_blue_smooth, T_blue_smooth_std = fit_hist(v_blue_smooth, bin_borders, MB_2D, [1.])
    T_red_smooth, T_red_smooth_std = fit_hist(v_red_smooth, bin_borders, MB_2D, [1.])
    print("Smooth Trajectories")
    print(f"Blue Particles σ: {T_blue_smooth[0]} ± {T_blue_smooth_std[0]}")
    print(f"Red Particle σ: {T_red_smooth[0]} ± {T_red_smooth_std[0]}")

In [ ]:
if plot_verb:
    fig, (ax, ax1) = plt.subplots(2, 1, figsize = (8, 5))
    ax.hist(v_blue, bins = bin_borders, **default_kwargs_blue)
    ax.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, T_blue), 'k-', label = f"$T = {T_blue[0]:.2f} \pm {T_blue_std[0]:.2f}$")
    ax.set(title = "Blue particles velocity distribution", xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.45))
    ax.legend()

    ax1.hist(v_red, bins = bin_borders, **default_kwargs_red)
    ax1.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, T_red), 'k-', label = f"$T = {T_red[0]:.2f} \pm {T_red_std[0]:.2f}$")
    ax1.set(title = "Red particle velocity distribution", xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.45))
    ax1.legend()

    plt.suptitle("Raw trajectories")
    plt.tight_layout()
    if save_verb: plt.savefig(f"./{res_path}/speed_distribution/speed_raw.png", )
    if show_verb: 
        plt.show()
    else:
        plt.close()

    if smooth_verb:
        fig, (ax, ax1) = plt.subplots(2, 1, figsize = (8, 5))
        ax.hist(v_blue_smooth, bins = bin_borders, **default_kwargs_blue)
        ax.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, T_blue_smooth), 'k-', 
                    label = f"$T = {T_blue_smooth[0]:.2f} \pm {T_blue_smooth_std[0]:.2f}$")
        ax.set(title = "Blue particles velocity distribution", xlabel = f"speed [{units}]", 
                    ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.3))
        ax.legend()

        ax1.hist(v_red_smooth, bins = bin_borders, **default_kwargs_red)
        ax1.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, T_red_smooth), 'k-', 
                    label = f"$T = {T_red_smooth[0]:.2f} \pm {T_red_smooth_std[0]:.2f}$")
        ax1.set(title = "Red particle velocity distribution", xlabel = f"speed [{units}]",
                    ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.3))
        ax1.legend()
        plt.suptitle("Smooth trajectories")
        plt.tight_layout()
        if save_verb: plt.savefig(f"./{res_path}/speed_distribution/speed_smooth.png", )
        if show_verb: 
            plt.show()
        else:
            plt.close()

## Windowed Analysis

In [ ]:
print("\n Windowed speed distribution Analysis")
v_blue_wind, v_red_wind = speed_windowed(nDrops, nSteps, startFrames, endFrames, red_particle_idx, rawTrajs, v_step, fps)
blue_fit_wind = np.ones((nSteps, 2))
red_fit_wind = np.ones((nSteps, 2))

for k in range(nSteps):
    blue_fit_wind[k, 0], blue_fit_wind[k, 1] = fit_hist(v_blue_wind[k], bin_borders, MB_2D, [1.])
    red_fit_wind[k, 0], red_fit_wind[k, 1] = fit_hist(v_red_wind[k], bin_borders, MB_2D, [1.])

if smooth_verb:
    v_blue_wind_smooth, v_red_wind_smooth = speed_windowed(nDrops, nSteps, startFrames, endFrames, red_particle_idx, smoothTrajs, v_step)
    blue_fit_wind_smooth = np.ones((nSteps, 2))
    red_fit_wind_smooth = np.ones((nSteps, 2))

    for k in range(nSteps):
        blue_fit_wind_smooth[k, 0], blue_fit_wind_smooth[k, 1] = fit_hist(v_blue_wind_smooth[k], bin_borders, MB_2D, [1.])
        red_fit_wind_smooth[k, 0], red_fit_wind_smooth[k, 1] = fit_hist(v_red_wind_smooth[k], bin_borders, MB_2D, [1.])

In [ ]:
r2_blue = np.zeros(nSteps)
r2_red = np.zeros(nSteps)
for step in range(nSteps):
    y = np.histogram(v_blue_wind[step], bins = bin_borders, density = True)[0]
    y_fit = MB_2D(bin_centers, blue_fit_wind[step, 0])
    ss_res = np.sum((y - y_fit) ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    r2_blue[step] = 1 - (ss_res / ss_tot)
    y = np.histogram(v_red_wind[step], bins = bin_borders, density = True)[0]
    y_fit = MB_2D(bin_centers, red_fit_wind[step, 0])
    ss_res = np.sum((y - y_fit) ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    r2_red[step] = 1 - (ss_res / ss_tot)

### PLOT

In [ ]:
if plot_verb:   
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(startFrames/fps, r2_blue, 'b')
    ax.grid(linewidth = 0.2)
    ax.set(xlabel = "time [s]", ylabel = "R2", title = "R2 of the fit of the velocity distribution")
    if save_verb: plt.savefig(f"./{res_path}/speed_distribution/r2_blue.png", bbox_inches='tight')
    if show_verb:
        plt.show()
    else:
        plt.close()

    # Effetcive Temperature plot
    fig, (ax, ax1) = plt.subplots(2, 1, figsize = (8, 4), sharex=True)
    ax.errorbar(startFrames/10, blue_fit_wind[:, 0], yerr = blue_fit_wind[:, 1], fmt = 'b', label="blue particles")
    ax.set(ylabel = "T [??]", ylim = (1, 5), title = "Effective Temperature - Raw Trajectories")
    ax.legend()
    ax.grid()
    ax1.errorbar(startFrames/10, red_fit_wind[:, 0], yerr = red_fit_wind[:, 1], fmt = 'r', label="red particles")
    ax1.set(xlabel = "Window Time [s]", ylabel = "T [??]")
    ax1.legend()
    ax1.grid()
    plt.tight_layout()
    if save_verb: plt.savefig(f'./{res_path}/speed_distribution/T_eff_raw.png', bbox_inches='tight')
    if show_verb:
        plt.show()
    else:
        plt.close()

    if smooth_verb:
        fig, (ax, ax1) = plt.subplots(2, 1, figsize = (8, 4), sharex=True)
        ax.errorbar(startFrames/10, blue_fit_wind_smooth[:, 0], yerr = blue_fit_wind_smooth[:, 1], fmt = 'b', label="blue particles")
        ax.set(ylabel = "T [??]", ylim = (1, 5), title = "Effective Temperature - Smooth Trajectories")
        ax.legend()
        ax.grid()

        ax1.errorbar(startFrames/10, red_fit_wind_smooth[:, 0], yerr = red_fit_wind_smooth[:, 1], fmt = 'r', label="red particles")
        ax1.set(xlabel = "Window Time [s]", ylabel = "T [??]")
        ax1.legend()
        ax1.grid()
        plt.tight_layout()
        if save_verb: plt.savefig(f'./{res_path}/speed_distribution/T_eff_smooth.png', bbox_inches='tight')
        if show_verb:
            plt.show()
        else:
            plt.close()

        
    if animated_plot_verb:
        fig, (ax, ax1) = plt.subplots(2, 1, figsize=(8, 5))
        anim_running = True

        def onClick(event):
            global anim_running
            if anim_running:
                ani.event_source.stop()
                anim_running = False
            else:
                ani.event_source.start()
                anim_running = True


        def prepare_animation(bar_container, bar_container2):
            def animate(frame):
                # update titles
                title.set_text(f"Raw Trajectories - window {startFrames[frame]/10} - {endFrames[frame]/10} seconds")
                #title2.set_text(f"Red particle velocity pdf {startFrames[frame]/10} - {endFrames[frame]/10} seconds")

                # update histogram 1
                n, _ = np.histogram(v_blue_wind[frame], bin_borders, density = True)
                for count, rect in zip(n, bar_container.patches):
                    rect.set_height(count)
                
                line.set_ydata(MB_2D(x_interval_for_fit, blue_fit_wind[frame, 0]))

                # update histogram 2
                n2, _ = np.histogram(v_red_wind[frame], bin_borders, density = True)
                for count2, rect2 in zip(n2, bar_container2.patches):
                    rect2.set_height(count2)
                    
                line1.set_ydata(MB_2D(x_interval_for_fit, red_fit_wind[frame, 0]))

                return bar_container.patches, bar_container2.patches
            return animate

        _, _, bar_container = ax.hist(v_blue_wind[0], bin_borders, **default_kwargs_blue, label="blue particles")
        title = ax.set_title(f"Raw Trajectories - window {startFrames[0]/10} - {endFrames[0]/10} seconds")
        line, = ax.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, blue_fit_wind[0, 0]), label='fit')
        ax.set(xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.5))
        ax.legend()

        _, _, bar_container2 = ax1.hist(v_red_wind[0], bin_borders,  **default_kwargs_red, label="red particle")
        #title2 = ax1.set_title(f"Red particle velocity pdf {startFrames[0]/10} - {endFrames[0]/10} seconds")
        line1, = ax1.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, red_fit_wind[0, 0]), label='fit')
        ax1.set(xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.5))
        ax1.legend()

        plt.tight_layout()
        fig.canvas.mpl_connect('button_press_event', onClick)
        ani = matplotlib.animation.FuncAnimation(fig, prepare_animation(bar_container, bar_container2), nSteps, repeat=True, blit=False)
        if save_verb: ani.save(f'./{res_path}/speed_distribution/speed_raw_wind.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
        if show_verb: 
            plt.show()
        else:
            plt.close()

        if smooth_verb:
            # smooth trajectories
            fig, (ax, ax1) = plt.subplots(2, 1, figsize=(8, 5))
            anim_running = True

            def onClick(event):
                global anim_running
                if anim_running:
                    ani.event_source.stop()
                    anim_running = False
                else:
                    ani.event_source.start()
                    anim_running = True


            def prepare_animation(bar_container, bar_container2):
                def animate(frame):
                    # update titles
                    title.set_text(f"Smooth Trajectories - window {startFrames[frame]/10} - {endFrames[frame]/10} seconds")
                    #title2.set_text(f"Red particle velocity pdf {startFrames[frame]/10} - {endFrames[frame]/10} seconds")

                    # update histogram 1
                    n, _ = np.histogram(v_blue_wind_smooth[frame], bin_borders, density = True)
                    for count, rect in zip(n, bar_container.patches):
                        rect.set_height(count)
                    
                    line.set_ydata(MB_2D(x_interval_for_fit, blue_fit_wind_smooth[frame, 0]))

                    # update histogram 2
                    n2, _ = np.histogram(v_red_wind_smooth[frame], bin_borders, density = True)
                    for count2, rect2 in zip(n2, bar_container2.patches):
                        rect2.set_height(count2)
                        
                    line1.set_ydata(MB_2D(x_interval_for_fit, red_fit_wind_smooth[frame, 0]))

                    return bar_container.patches, bar_container2.patches
                return animate

            _, _, bar_container = ax.hist(v_blue_wind_smooth[0], bin_borders, **default_kwargs_blue, label="blue particles")
            title = ax.set_title(f"Raw Trajectories - window {startFrames[0]/10} - {endFrames[0]/10} seconds")
            line, = ax.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, blue_fit_wind_smooth[0, 0]), label='fit')
            ax.set(xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.5))
            ax.legend()

            _, _, bar_container2 = ax1.hist(v_red_wind_smooth[0], bin_borders,  **default_kwargs_red, label="red particle")
            #title2 = ax1.set_title(f"Red particle velocity pdf {startFrames[0]/10} - {endFrames[0]/10} seconds")
            line1, = ax1.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, red_fit_wind_smooth[0, 0]), label='fit')
            ax1.set(xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.5))
            ax1.legend()

            plt.tight_layout()
            fig.canvas.mpl_connect('button_press_event', onClick)
            ani = matplotlib.animation.FuncAnimation(fig, prepare_animation(bar_container, bar_container2), nSteps, repeat=True, blit=False)
            if save_verb: ani.save(f'./{res_path}/speed_distribution/speed_smooth_wind.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
            if show_verb: 
                plt.show()
            else:
                plt.close()